This notebook add to each species the total emissions coming from the sum of emissions of all sectors.

In [1]:
import xarray as xr
import os
import numpy as np
import pandas as pd

In [2]:
#get emissions
emi_dir = '/geos/d21/s1878599/edgarv5_process/monthly_all_sectors/'
save_dir='/geos/d21/s1878599/edgarv5_process/monthly_all_sectors_with_tot/'
#create save directory if missing.
if not os.path.isdir(save_dir):
    !mkdir -p $save_dir

## Add total emissions for each species

In [3]:
def add_total_emi(ds):
    sectors= [da.name for da in ds.data_vars.values()]
    ds['TOTAL']=xr.zeros_like(ds['ENE']) # create empty datarray for total
    ds['TOTAL'].attrs=ds['ENE'].attrs
    
    for s in sectors:
        ds['TOTAL']=ds['TOTAL']+ds[s] 

In [4]:
# calc total for each pollutant

for f in os.listdir(emi_dir):
    vname=f.split('_')[3]
    print(vname)
    
    ds=xr.open_dataset(emi_dir+f)
    ds_new=add_total_emi(ds) 
    

    ds.to_netcdf(save_dir+'monthly_v50_2015_'+vname+'_.0.1x0.1.nc',format='NETCDF3_64BIT') #save new file.

C2H5OH
CO
CH3OH
CH3COOH
MEK
TOLUENE
BIGENE
CH3COCH3
CH2O
C2H6
NMVOC
NOx
BENZENE
XYLENES
HCOOH
PM10
SO2
C3H8
OC
C2H2
C3H6
CH3CHO
BC
NH3
PM2.5
C2H4
BIGALK


## Check if total is calculated correctly

In [6]:
t=xr.open_dataset(save_dir+'monthly_v50_2015_C2H5OH_.0.1x0.1.nc')

In [7]:
t

<xarray.Dataset>
Dimensions:           (lat: 1800, lon: 3600, time: 12)
Coordinates:
  * lat               (lat) float32 -89.95 -89.85 -89.75 ... 89.75 89.85 89.95
  * lon               (lon) float32 0.05 0.15 0.25 0.35 ... 359.75 359.85 359.95
  * time              (time) int32 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    AWB               (time, lat, lon) float32 ...
    ENE               (time, lat, lon) float32 ...
    IND               (time, lat, lon) float32 ...
    PPA               (time, lat, lon) float32 ...
    PRO               (time, lat, lon) float32 ...
    RCO               (time, lat, lon) float32 ...
    SWD               (time, lat, lon) float32 ...
    TNR-Aviation-CDS  (time, lat, lon) float32 ...
    TNR-Aviation-CRS  (time, lat, lon) float32 ...
    TNR-Aviation-LTO  (time, lat, lon) float32 ...
    TNR-Other         (time, lat, lon) float32 ...
    TNR-Ship          (time, lat, lon) float32 ...
    REF-TRF           (time, lat, lon) float32 ...
    TRO               (time, lat, lon) float32 ...
    FFF               (time, lat, lon) float32 ...
    TOTAL             (time, lat, lon) float32 ...
Attributes:
    title:    Monthly emissions of C2H5OH kg m-2 s-1

In [8]:
t['TOT_CALC']=t['AWB']+t['ENE']+t['FFF']+t['PPA']+t['IND']+t['PRO']+t['RCO']+t['REF-TRF']+t['SWD']+\
t['TNR-Aviation-CDS']+t['TNR-Aviation-CRS']+t['TNR-Aviation-LTO']+t['TNR-Other']+t['TNR-Ship']+t['TRO']

xr.testing.assert_allclose( t['TOT_CALC'],  t['TOTAL']) 